# Table 2.

Global top-of-atmosphere shortwave ($CRE_{SW}$), longwave ($CRE_{LW}$), and total ($CRE$) cloud radiative effect ($W~m^{-2}$) in Hab1 and Hab2 simulations.

In [1]:
# Data analysis and viz libraries
import pandas as pd
import xarray as xr

In [2]:
# Local modules
import mypaths
import names
from calc import spatial_mean
from load_thai import LOAD_CONF

In [3]:
def cre_toa(ds, model_key, kind="total"):
    r"""
    Calculate domain-average TOA cloud radiative effect (CRE).

    .. math::
        CRE_{TOA} = F_{up,clear-sky} - F_{up,all-sky}

    Parameters
    ----------
    ds: xarray.Dataset
        Input dataset containing relevant variables.
    model_key: str,
        Model name.
    kind: str, optional
        Shortwave ('sw'), longwave ('lw'), or 'total' CRE.

    Returns
    -------
    xarray.DataArray
    """
    name = f"toa_cloud_radiative_effect_{kind}"
    model_names = getattr(names, model_key.lower())
    if kind == "sw":
        if model_key == "ExoCAM":
            out = -(ds[model_names.toa_net_sw_cs] - ds[model_names.toa_net_sw])
        if model_key == "LMDG":
            out = -(ds[model_names.toa_net_sw_cs] - ds[model_names.toa_net_sw])
        elif model_key == "ROCKE3D":
            out = ds[model_names.toa_crf_lw]  # FIXME
        elif model_key == "UM":
            out = ds[model_names.toa_osr_cs] - ds[model_names.toa_osr]
    elif kind == "lw":
        if model_key == "ExoCAM":
            out = ds[model_names.toa_net_lw_cs] - ds[model_names.toa_net_lw]
        if model_key == "LMDG":
            out = ds[model_names.toa_olr_cs] - ds[model_names.toa_olr]
        elif model_key == "ROCKE3D":
            out = ds[model_names.toa_crf_lw]
        elif model_key == "UM":
            out = ds[model_names.toa_olr_cs] - ds[model_names.toa_olr]
    elif kind == "total":
        sw = cre_toa(ds, model_key, "sw")
        lw = cre_toa(ds, model_key, "lw")
        out = sw + lw
        out = out.rename(name)
        return out

    out = out.rename(name)
    return out

## Choose cases

In [4]:
THAI_cases = ["Hab1", "Hab2"]

## Loading the data

Load the time-averaged data previously preprocessed.

In [5]:
# Load data
datasets = {}  # Create an empty dictionary to store all data
# Loop over THAI cases (this loop can swapped places with the loop below if needed)
for THAI_case in THAI_cases:
    # for each of the THAI cases, create a nested directory for models
    datasets[THAI_case] = {}
    for model_key in LOAD_CONF.keys():
        model_names = getattr(names, model_key.lower())
        datasets[THAI_case][model_key] = xr.open_dataset(
            mypaths.datadir / model_key / f"{THAI_case}_time_mean_{model_key}.nc"
        )
        # datasets[THAI_case][model_key] = LOAD_CONF[model_key]["loader"](THAI_case)

In [6]:
diags = {
    "$CRE_{SW}$": lambda ds, model_key: cre_toa(ds, model_key, kind="sw"),
    "$CRE_{LW}$": lambda ds, model_key: cre_toa(ds, model_key, kind="lw"),
    "$CRE$": lambda ds, model_key: cre_toa(ds, model_key, kind="total"),
}

In [7]:
results = {}
for THAI_case in THAI_cases:
    dfs = {}
    for model_key in LOAD_CONF.keys():
        _data = {}
        for vrbl, _calc in diags.items():
            _data[vrbl] = float(
                spatial_mean(_calc(datasets[THAI_case][model_key], model_key)).values
            )
        dfs[model_key] = pd.DataFrame(
            _data, index=pd.Index(name="GCM", data=[model_key])
        )
    results[THAI_case] = pd.concat(dfs, axis="index").droplevel(0).T
df = pd.concat(results).T

In [8]:
print(
    df.to_latex(
        formatters={k: lambda x: f"{x:.1f}" for k in df.columns},
        column_format="l" + "c" * len(diags) * len(THAI_cases),
        escape=False,
    )
)

\begin{tabular}{lcccccc}
\toprule
{} & \multicolumn{3}{l}{Hab1} & \multicolumn{3}{l}{Hab2} \\
{} & $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ & $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ \\
GCM     &            &            &       &            &            &       \\
\midrule
ExoCAM  &      -37.3 &       15.2 & -22.1 &      -21.7 &       13.1 &  -8.6 \\
LMDG    &      -21.5 &        5.0 & -16.5 &      -31.8 &        3.2 & -28.6 \\
ROCKE3D &        9.5 &        9.5 &  19.0 &        8.5 &        8.5 &  17.1 \\
UM      &      -48.1 &       10.1 & -38.0 &      -27.3 &        8.3 & -19.0 \\
\bottomrule
\end{tabular}

